In [1]:
!pip install -q transformers datasets

In [6]:
#pip install fsspec==2025.3.2
!pip install -q transformers datasets accelerate bitsandbytes peft trl


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [4]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="mirshan_dataset.jsonl", split="train")
dataset = dataset.train_test_split(test_size=0.1)

Generating train split: 0 examples [00:00, ? examples/s]

In [30]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "openai-community/gpt2"  # veya `ai4bharat/indic-bert` ya da `TRC-GPT2`
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [31]:
#Peft ve Lora ile verimli eğitim
from peft import get_peft_model, LoraConfig, TaskType

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=4,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none"
)
model = get_peft_model(model, config)

In [36]:
from transformers import TrainingArguments
from trl import SFTTrainer

args = TrainingArguments(
    output_dir="./mirshan-llm",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=100,
    save_total_limit=2
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=args
)
trainer.train()


Converting train dataset to ChatML:   0%|          | 0/7 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/7 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/7 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:630: UserWarning: Mismatch between tokenized prompt and the start of tokenized prompt+completion. This may be due to unexpected tokenizer behavior, whitespace issues, or special token handling. Verify that the tokenizer is processing text consistently.
  warnings.warn(


Truncating train dataset:   0%|          | 0/7 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss


TrainOutput(global_step=12, training_loss=4.844241301218669, metrics={'train_runtime': 56.6581, 'train_samples_per_second': 0.371, 'train_steps_per_second': 0.212, 'total_flos': 2131279248384.0, 'train_loss': 4.844241301218669})

In [37]:
model.save_pretrained("mirshan-model")


In [20]:
tokenizer.save_pretrained("mirshan-model")


('mirshan-model/tokenizer_config.json',
 'mirshan-model/special_tokens_map.json',
 'mirshan-model/vocab.json',
 'mirshan-model/merges.txt',
 'mirshan-model/added_tokens.json',
 'mirshan-model/tokenizer.json')

In [42]:
#test model

In [40]:
import torch

def generate_response(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"]

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_p=0.95,
            temperature=0.8
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Örnek:
test_prompt = "this is totally weird "
response = generate_response(test_prompt)
print(response)


this is totally weird irl. i think you're the best of them. i'm sorry i'm the only one i really really have no fear. i don't know if that will ever be the best.

anon 541 - I don't know how you will live. i do know how you will have to live. i do you want to live. you want to live.

anon 534 - 18 points

anon 434

anon 534 - 5
